# Challenge IA Générative : Construire un moteur de recommandation avec LangChain

### Définition du problème :

#### Le problème à résoudre

“Comment recommander des films ou séries pertinents quand l’utilisateur ne connaît pas les titres, mais seulement le type d’histoire qu’il cherche ?”

Un utilisateur ne tape pas :

"Naruto"

Il tape plutôt :

"anime d’action avec des animaux"

Ce type de requête est :

* vague

* subjective

* basée sur le sens, pas sur des mots-clés exacts

🎯 L'objectif du notebook est de créer un système capable de :

* comprendre le sens d’une requête

* parcourir une base d’animes

* retourner les animes les plus proches sémantiquement

### Ce qu'il faut faire : Préparer la matière première (les données)

Avant LangChain, il y a la donnée brute.

🔹 Source de données

Nous utilisons un fichier CSV contenant :

* le nom de l’anime

* son synopsis

* ses genres

Mais ces informations sont dispersées sur plusieurs colonnes.

👉 Or, un modèle de langage comprend mieux :

* un texte continu

* structuré comme un paragraphe

### Présentation de Langchain

LangChain n’est PAS un modèle de langage. C’est un framework d’orchestration pour applications IA.

Il sert à connecter :

* des LLM (OpenAI, Hugging Face, etc.)

* des sources de données (FAISS, Chroma, SQL…)

* des chaînes de raisonnement

* des outils

### Les briques fondamentales de LangChain

| Brique                   | Rôle principal       | Description simple                                               | Exemples                                      |
| ------------------------ | -------------------- | ---------------------------------------------------------------- | --------------------------------------------- |
| **LLM (Language Model)** | Générer du texte     | Modèle de langage qui comprend une entrée et produit une réponse | OpenAI (ChatGPT), Hugging Face, Ollama, Azure |
| **Embeddings**           | Comprendre le sens   | Transforme le texte en vecteurs numériques pour comparer le sens | Hugging Face Embeddings, OpenAI Embeddings    |
| **Vector Stores**        | Stocker & rechercher | Stocke les vecteurs et permet la recherche par similarité        | FAISS, Chroma, Pinecone                       |
| **Chains (chaînes)**     | Orchestration        | Enchaîne plusieurs étapes (prompt, recherche, génération)        | `LLMChain`, `RetrievalQA`                     |
| **Agents** *(avancé)*    | Décision autonome    | Le LLM choisit dynamiquement des actions, outils ou API          | Agents LangChain, Tool Calling                |


#### Exemple concret pour chaque brique LangChain

On part d’un cas réel :
🎬 Recommander un film d’animation d’action

1️⃣ LLM (Language Model) 👉 Génère la réponse finale


```` python
prompt = "Recommande un film d'animation d'action"
llm(prompt)
````

Rôle : écrire une réponse compréhensible pour l’humain.



2️⃣ Embeddings 👉 Comprendre le sens de la requête

```` python
embeddings.embed_query("film d'animation action")
````

Rôle : transformer du texte en nombres comparables.



3️⃣ Vector Store (FAISS / Chroma) 👉 Retrouver les documents pertinents
```` python
docsearch.similarity_search("film d'animation action", k=2)
````

Rôle : trouver les genres ou descriptions les plus proches.



4️⃣ Chains (RetrievalQA) 👉 Enchaîner recherche + génération
```` python
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=docsearch.as_retriever()
)

qa.invoke("Recommande un film d'animation d'action")
````
Rôle : automatiser le pipeline complet.



5️⃣ Agents (avancé) 👉 Décider quoi faire
```` python
agent.run("Trouve un film d'action et explique pourquoi")
````

Rôle : le modèle choisit s’il doit chercher, calculer, appeler une API.

### En résumé

LLM → “Je parle”

Embeddings → “Je comprends le sens”

Vector Store → “Je retrouve l’info”

Chains → “J’enchaîne les étapes”

Agents → “Je décide quoi faire”

## Installations

In [1]:
!python -m venv genai_env
!genai_env\Scripts\activate    # Windows

!pip install langchain langchain-community langchain-openai chromadb openai tiktoken --quiet


Error: Command '['/content/genai_env/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: genai_envScriptsactivate: command not found


In [2]:
import langchain
print(langchain.__version__)

1.2.0


In [3]:
!pip install --upgrade langchain
!pip install --upgrade langchain_experimental


**Loading Libraries**

In [4]:
# Quel module permet la manipulation et l'analyse des données ? Indiquez le nom correct du module.
import pandas as pd

# Quel est le nom du module qu'on utilise souvent pour gérer des fonctionnalités liées aux tokens mais qui semble ici être une erreur de frappe ?
import tiktoken as tiktoken

# Quel module de la bibliothèque standard Python est utilisé pour interagir avec le système d'exploitation ?
import os as os

# Comment importez-vous la classe 'RetrievalQA' depuis le module 'langchain.chains' ?
# aujourd’hui RetrievalQA est remplacé par une chaîne RAG explicite, construite avec :
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Comment importez-vous 'TextLoader' depuis 'langchain.document_loaders' ?
# from langchain.document_loaders import TextLoader

# Quelle classe 'CharacterTextSplitter' doit être importée depuis 'langchain.text_splitter' ?
#from langchain.schema import Document

# D'où importez-vous 'Chroma' dans le module 'langchain.vectorstores' ?
from langchain_community.vectorstores import Chroma

# Comment importez-vous 'CSVLoader' depuis 'langchain.document_loaders.csv_loader' ?
# from langchain.document_loaders.text import TextLoader



In [5]:
import sys
print(sys.executable)


/usr/bin/python3


**Data Preprocessing**

### Objectif : Fusion des informations textuelles

In [7]:
# Chargez un fichier CSV dans un DataFrame. Remplacez les parties manquantes pour compléter le chemin du fichier.
anime = pd.read_csv('anime_with_synopsis.csv')

# Affichez les premières lignes du DataFrame. Complétez le code pour afficher les cinq premières lignes.
anime.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


In [8]:
# Combinez plusieurs colonnes dans une seule en utilisant une fonction lambda pour formater la chaîne de caractères. Remplissez les parties manquantes.
anime['combined_info'] = anime.apply(lambda row: f"Title: {row['Name']}. Overview: {row['sypnopsis']} Genres: {row['Genres']}", axis=1)

# Affichez le premier élément de la nouvelle colonne créée. Complétez les parties manquantes.
anime['combined_info'][0]

'Title: Cowboy Bebop. Overview: In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind. The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as "Cowboys." The ragtag team aboard the spaceship Bebop are two such individuals. Mellow and carefree Spike Spiegel is balanced by his boisterous, pragmatic partner Jet Black as the pair makes a living chasing bounties and collecting rewards. Thrown off course by the addition of new members that they meet in their travels—Ein, a genetically engineered, highly intelligent Welsh Corgi; femme fatale Faye Valentine, an enigmatic trickster with memory loss; and the strange computer whiz kid Edward Wong—the crew embarks on thrilling adventures that unravel each member\'s dark and mysterious past little by little. Well-balanced with high density action and light-hearted comedy, Cowboy Bebo

Nous transformons chaque ligne du CSV en une mini-description complète.
Cette description sera l’unité de base comprise par le modèle sémantique.

#### Sauvegarde du nouveau jeu de données

LangChain ne travaille pas directement sur des DataFrames pandas

Il attend des fichiers ou des documents textuels

In [9]:
#Save processed dataset - combined_info for Langchain
anime[['combined_info']].to_csv('anime_updated.csv', index=False)

In [10]:
# Chargez ce nouveau fichier CSV dans un DataFrame. Remplacez les parties manquantes pour spécifier le chemin correct du fichier.
pd.read_csv('anime_updated.csv')

,combined_info
0,Title: Cowboy Bebop. Overview: In the year 207...
1,Title: Cowboy Bebop: Tengoku no Tobira. Overvi...
2,Title: Trigun. Overview: Vash the Stampede is ...
3,Title: Witch Hunter Robin. Overview: ches are ...
4,Title: Bouken Ou Beet. Overview: It is the dar...
...,...
16209,Title: Daomu Biji Zhi Qinling Shen Shu. Overvi...
16210,Title: Mieruko-chan. Overview: ko is a typical...
16211,Title: Higurashi no Naku Koro ni Sotsu. Overvi...
16212,Title: Yama no Susume: Next Summit. Overview: ...


**Data Loader and Vector store using Langchain**

### Chargement et préparation des données : Transformer les données en Documents LangChain

Chaque ligne du CSV devient un Document.

Un Document contient :

* page_content → le texte

* metadata → des informations contextuelles

C’est cette abstraction qui permet ensuite la recherche, la traçabilité et l’explicabilité

In [19]:
from langchain_community.document_loaders import CSVLoader

# Chargez des données à partir d'un fichier CSV en utilisant 'CSVLoader'.
loader = CSVLoader(file_path="anime_updated.csv")
data = loader.load()



### Chargement des documents avec LangChain

In [16]:
!pip install langchain-community


In [17]:
import langchain
import langchain_community
print("LangChain OK")

LangChain OK


#### Découper pour mieux comprendre (Text Splitting):

Les modèles d’embeddings ont des limites :

* en taille

* en précision sur les longs textes

Solution : découper intelligemment

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [47]:
# Configurez un transformateur de texte pour diviser les documents en morceaux.
text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap= 200)
texts = text_splitter.split_documents(data)

Découpage récursif : On découpe chaque description en morceaux de texte qui :

* se chevauchent légèrement

* gardent le contexte narratif

Cela améliore la qualité des embeddings et donc des recommandations.

In [25]:
pip install -U langchain langchain-community langchain-core sentence-transformers faiss-cpu


#### Donner du sens aux textes (Embeddings)

À ce stade, nous avons :

* du texte propre

* bien structuré

* découpé

Mais l’ordinateur ne comprend toujours pas le sens.

Il faut transformer le texte en vecteurs numériques.

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

Ce modèle transforme chaque morceau de texte en un vecteur.
Deux textes proches en signification auront des vecteurs proches dans l’espace.

Exemple conceptuel :

“anime d’action avec des animaux”

“aventures de créatures combattantes”

--> vecteurs très proches

### Mémoriser le sens : Base vectorielle (FAISS)

* Stocker les vecteurs
* Permettre la recherche sémantique

#### Création de la base vectorielle

In [ ]:
from langchain_community.vectorstores import FAISS

docsearch = FAISS.from_documents(texts, embeddings)

# FAISS agit comme une mémoire sémantique.
# Il ne stocke pas des mots, mais des idées mathématiques.

### Interroger le système (la requête utilisateur)

### Recherche sémantique (le cœur du RAG)

1. La requête est transformée en vecteur

2. FAISS compare ce vecteur aux documents

3. Il retourne les plus proches sémantiquement

In [33]:
# Définissez une requête pour rechercher un anime d'action.
query = "anime d'action avec science-fiction"

# Utilisez l'instance 'qa' pour obtenir une réponse à votre requête.
result = rag_chain.invoke(query)
result


# La requête est traitée exactement comme les documents : transformée en vecteur et comparée à tous les autres vecteurs

"Tu es un expert en anime.\nUtilise uniquement les genres suivants pour répondre :\n\n[Document(id='6a867e89-8947-4dd4-8c25-f026a56f34c6', metadata={}, page_content='Action, Sci-Fi, Psychological, Drama')]\n\nQuestion : anime d'action avec science-fiction\nRéponse :"

In [26]:
from langchain_core.documents import Document

docs = [
    Document(page_content="Action, Sci-Fi, Psychological, Drama"),
    Document(page_content="Action, Adventure, Comedy, Shounen"),
    Document(page_content="Comedy, Romance, School, Shounen"),
]

# Recherche de similarité
# Le système ne cherche pas des mots identiques. Il cherche des proximité de sens.

results = docsearch.similarity_search(
    "film d'animation action science-fiction",
    k=1
)

# Affichage des résultats
print(results[0].page_content)


/tmp/ipython-input-3054594312.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Action, Sci-Fi, Psychological, Drama


#### Donner une voix au système (LLM local)

Jusqu’ici, notre système sait :

* chercher des documents

* comparer des significations

* retrouver des animes pertinents

❌ Mais il ne sait pas encore formuler une réponse humaine.

Nous avons besoin d’un modèle de génération de texte.

In [50]:
from langchain_huggingface import HuggingFacePipeline   # HuggingFacePipeline : adaptateur LangChain → transforme un modèle HF en LLM compatible LangChain
from transformers import pipeline    # transformers.pipeline : interface simple pour charger un modèle NLP



LangChain ne génère pas de texte, il orchestre des modèles qui savent le faire.

Pourquoi FLAN-T5 ?

* entraîné pour suivre des instructions

* très adapté au RAG

* fonctionne localement

In [49]:
# Création du pipeline de génération :
hf_pipeline = pipeline(
    "text-generation",    # text-generation : tâche NLP
    model="google/flan-t5-base",    # google/flan-t5-base : modèle instruction-tuned
    max_new_tokens=256,     # max_new_tokens=256 : longueur max de la réponse
    temperature=0.7    # temperature=0.7 : créativité contrôlée
)

# Conversion en LLM LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'FlexOlmoF

À partir de maintenant, LangChain peut parler.
Ce LLM sera la voix finale de notre moteur de recommandation.

### Recherche sémantique simple (sans génération)

Avant de générer une réponse, vérifions que la recherche fonctionne.

In [51]:
# Définissez une requête de recherche pour trouver un film d'animation d'action.
query = "film d'animation action aventure science-fiction"

Cette requête n’est pas un titre, exprime une intention et repose sur le sens

#### Recherche de similarité


In [27]:
# Effectuez une recherche de similarité dans la base de données 'docsearch' pour trouver le document le plus pertinent.
docs = docsearch.similarity_search(query, k=1)   # k signifie le nombre de résultats retournés (k=1 → le document le plus pertinent, k=10 → les 10 meilleurs résultats)
docs


[Document(id='6a867e89-8947-4dd4-8c25-f026a56f34c6', metadata={}, page_content='Action, Sci-Fi, Psychological, Drama')]

* similarity_search(...). Cette méthode transforme la requête en vecteur, compare ce vecteur à ceux des documents stockés et calcule une distance

* (cosine similarity, euclidienne, etc.). Il retourne ensuite les documents les plus proches sémantiquement


### Inspection du résultat

Nous vérifions ici que la base vectorielle comprend bien la requête,
avant même d’impliquer un modèle génératif.

In [28]:
docs[0].page_content


'Action, Sci-Fi, Psychological, Drama'

### Du retrieval à la recommandation (QA Retrieval)

Nous voulons maintenant poser une question, récupérer des documents et générer une réponse

C’est le principe du RAG.

#### Installation des dépendances

LangChain est modulaire : chaque brique est installée séparément.

In [29]:
pip install -U langchain-huggingface transformers sentence-transformers


Pourquoi RetrievalQA est abandonné :
* boîte noire

* difficile à débugger

* rigide

#### Pourquoi LangChain a supprimé RetrievalQA ?

| Ancien               | Nouveau            |
| -------------------- | ------------------ |
| `RetrievalQA`        | LCEL               |
| classes magiques     | chaînes explicites |
| difficile à debugger | transparent        |
| rigide               | composable         |


#### Nouvelle approche (LCEL)

LangChain moderne = chaînes explicites
LCEL = LangChain Expression Language
👉 Tout est visible, composable, explicable.

In [ ]:
# Depuis LangChain 1.x (2024–2025) : RetrievalQA est DÉPRÉCIÉ, langchain.chains n’existe plus comme avant
# LangChain a basculé vers : Runnable, LCEL (LangChain Expression Language)
# On oublie RetrievalQA, on fait un RAG moderne avec LCEL, 100 % compatible avec Hugging Face
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


### Le prompt (le cerveau du RAG)

🧠 Pourquoi un prompt ?

Le LLM ne sait pas quoi faire sans instructions et peut halluciner sans contrainte

Le prompt :

* définit son rôle

* limite ses sources

* structure la réponse

In [52]:
prompt = PromptTemplate.from_template(
    """Tu es un expert en anime.
Utilise uniquement les genres suivants pour répondre :

{context}

Question : {question}
Réponse :"""
)

### Le retriever : le moteur de recherche sémantique

Le retriever est une interface standard fournie par LangChain.
Son rôle est d’encapsuler la recherche vectorielle et de la rendre réutilisable dans différents pipelines.

Le retriever :

* reçoit une requête en langage naturel,

* transforme cette requête en vecteur,

* compare ce vecteur aux vecteurs stockés dans la base,

* retourne les documents les plus proches sémantiquement.

Ainsi, le retriever agit comme un pont entre la question utilisateur et les données.

In [53]:
retriever = docsearch.as_retriever(search_kwargs={"k": 1})
# L’argument k=1 indique que l’on souhaite récupérer un seul document, c’est-à-dire celui qui est jugé le plus pertinent par la recherche vectorielle.

#### Construction du RAG avec LCEL (LangChain Expression Language).
La chaîne RAG est construite à l’aide de LCEL (LangChain Expression Language).
Cette approche rend le pipeline explicite, lisible et facile à comprendre.


Dans notre cas, le prompt contient deux emplacements dynamiques :

* {context} : les documents récupérés par la recherche sémantique

* {question} : la question originale de l’utilisateur

In [32]:
rag_chain = (
    {
        "context": retriever,    # La question est envoyée au retriever : La question sert à effectuer une recherche sémantique.
      # Les documents récupérés deviennent la valeur de {context}.
        "question": RunnablePassthrough()  # La question originale est conservée sans modification.
      # Elle devient la valeur de {question}.
    }
    | prompt
    | llm
)




"Tu es un expert en anime.\nUtilise uniquement les genres suivants pour répondre :\n\n[Document(id='6a867e89-8947-4dd4-8c25-f026a56f34c6', metadata={}, page_content='Action, Sci-Fi, Psychological, Drama')]\n\nQuestion : anime d'action avec de la science-fiction\nRéponse :"

Le fonctionnement se déroule en plusieurs étapes successives :

**1. Arrivée de la question**

La chaîne reçoit une entrée unique :
la question formulée par l’utilisateur en langage naturel.

In [54]:
response = rag_chain.invoke("anime d'action avec de la science-fiction")
response

"You are a movie recommender system that help users to find anime that match their preferences. \nUse the following pieces of context to answer the question at the end. \nFor each question, take into account the context and the personal information provided by the user.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nAction, Sci-Fi, Psychological, Drama\n\nAction, Adventure, Comedy, Shounen\n\nComedy, Romance, School, Shounen\nThis is what we know about the user, and you can use this information to better tune your research:\nAge: 25\nGender: male\nQuestion: anime d'action avec de la science-fiction\nYour response:"

Ici, la chaîne reçoit une seule valeur en entrée *("anime d'action avec de la science-fiction")*

**2. Duplication logique de l’entrée**

LangChain exécute automatiquement :

In [35]:
# Récupérer les documents sources
source_docs = retriever.invoke(query)



'Action, Sci-Fi, Psychological, Drama'

Le résultat est une liste de documents qui représente le contexte.

LangChain a construit implicitement le dictionnaire suivant :

```` python
{
    "context": [Document(...), Document(...)],
    "question": "anime d'action avec de la science-fiction"
}

````

les documents deviennent {context}

la question reste {question}

## **Prompt Engineering**

Remplissage du prompt

Le prompt est complété automatiquement avec les valeurs obtenues à l’étape précédente.


#### **Premier template : PromptTemplate LangChain (injection dynamique)**

On crée un objet PromptTemplate qui permet de déclarer explicitement quelles variables seront injectées plus tard et permet à LangChain de remplir le prompt automatiquement

Ce prompt garantit aussi que {context} et {question} existent bien

In [36]:
# Premier template (RAG – contexte)

# Définir un modèle de prompt pour un système de recommandation de films
template = """You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason why the user might like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""

# Créez une instance de 'PromptTemplate'
PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)



**Deuxième Template** - Provinding additional user info in the context

Des informations utilisateur sont ajoutées au prompt.
La construction du texte du prompt se fait à l'aide de templates statiques.

In [37]:
# Définissez les différentes parties d'un modèle de prompt
template_prefix = """You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}"""

# Informations utilisateur ajoutées au prompt
user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}"""

# Fin du prompt (question / réponse)
template_suffix = """Question: {question}
Your response:"""


# Formattez 'user_info' avec des informations spécifiques sur l'utilisateur.
user_info = user_info.format(age=25, gender='male')
# À ce stade : {age} et {gender} sont déjà remplacés, ce texte devient statique pour la suite du RAG

# Combinez les trois parties du prompt pour former un prompt complet.
COMBINED_PROMPT = template_prefix + '\n' + user_info + '\n' + template_suffix
print(COMBINED_PROMPT)



You are a movie recommender system that help users to find anime that match their preferences. 
Use the following pieces of context to answer the question at the end. 
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
This is what we know about the user, and you can use this information to better tune your research:
Age: 25
Gender: male
Question: {question}
Your response:


Ce deuxième template définit le rôle du modèle (système de recommandation), impose des règles (ne pas halluciner) et contient déjà l’emplacement {context}.

### Remplissage automatique du prompt (au moment de l’exécution)

Ce remplissage se fait uniquement lors de l’exécution de la chaîne RAG.




In [40]:
from langchain_core.output_parsers import StrOutputParser

# PromptTemplate (prompt combiné)
PROMPT = PromptTemplate(
    template=COMBINED_PROMPT,
    input_variables=["context", "question"]
)

# Fonction de formatage des documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Création de la chaîne RAG moderne (LCEL)
rag_chain = (
    {
        "context": docsearch.as_retriever() | format_docs,
        "question": RunnablePassthrough(),
    }
    | PROMPT     # Cela signifie que LangChain appelle en interne : PROMPT.format(context=context,question=question)
    | llm
    | StrOutputParser()
)

# Requête
query = "anime d'action avec des animaux"

# Exécution
result = rag_chain.invoke(query)
print(result)




You are a movie recommender system that help users to find anime that match their preferences. 
Use the following pieces of context to answer the question at the end. 
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Action, Adventure, Comedy, Shounen

Action, Sci-Fi, Psychological, Drama

Comedy, Romance, School, Shounen
This is what we know about the user, and you can use this information to better tune your research:
Age: 25
Gender: male
Question: anime d'action avec des animaux
Your response:


In [42]:
# Affichez le détail du résultats
retriever = docsearch.as_retriever()
source_docs = retriever.invoke(query)

source_docs[0]
source_docs[0].page_content


'Action, Adventure, Comedy, Shounen'

### Résumé : Ce que contient exactement le prompt final

Le prompt final contient explicitement :

Le rôle du modèle
→ système de recommandation d’animes

Les contraintes de génération
→ ne pas inventer de réponse

Les informations autorisées
→ uniquement les documents récupérés (context)

Les informations utilisateur
→ âge, genre

La question utilisateur
→ injectée telle quelle

La structure de sortie
→ Your response:

### **Résumé** :

Templates : définissent la forme, les règles et la structure du prompt

PromptTemplate : injecte automatiquement les données au runtime

LLM : génère une réponse à partir du texte final uniquement

### 1. **Templates — Texte brut du prompt (niveau humain)**

Définition

* Les templates sont du texte Python classique (str) utilisé pour définir à l’avance la forme du prompt.

Ils servent à :

* définir le rôle du modèle

* fixer les règles de réponse

* organiser la structure

* prévoir des variables dynamiques ({context}, {question})

*Les templates sont des plans de prompt, pas des prompts exécutables.*

A ce stade, {context} et {question} sont des placeholders

❌ aucun document n’est injecté

❌ aucune question n’est connue

❌ le texte n’est pas utilisable par un LLM

### **2. PromptTemplate — Moteur de remplissage (niveau LangChain)**
📌 Pourquoi PromptTemplate existe

Un LLM ne comprend que du texte final. LangChain a donc besoin d’un mécanisme pour :

* savoir quelles variables remplacer

* injecter automatiquement les valeurs

* sécuriser le format du prompt

* connecter le prompt à une chaîne LCEL

C’est le rôle de PromptTemplate. PromptTemplate transforme un texte statique en objet exécutable.

*C'est le pont entre le texte statique et les données dynamiques.*


### **3. Injection automatique dans LCEL (runtime)**

**1ère étape : Calcul des variables**

* context ← documents récupérés et formatés

* question ← question utilisateur inchangée

**2ème étape : Remplissage du prompt**

```` python
final_prompt = PROMPT.format(
    context=context,
    question=question
)
````
Un seul texte final, prêt pour le modèle.

*Le PromptTemplate est évalué au moment de l’exécution, pas à la définition du code.*

### **4. LLM — Génération à partir du prompt final**

```` python
| llm
````

📌 Rôle du LLM

Le LLM (Flan-T5, GPT, etc.) :

reçoit uniquement le texte final

❌ ne connaît ni LangChain

❌ ne connaît ni les templates

❌ ne connaît ni le retriever

**Ce que voit le LLM**

```` python
You are a movie recommender system...
[documents injectés]

Question: anime d'action avec de la science-fiction
Your response:
````
Ce que fait le LLM :

1. lit les instructions

2. respecte les contraintes

3. génère du texte token par token

4. s’appuie uniquement sur le prompt fourni

### **Important (à retenir) :**

**Erreurs fréquentes à éviter**

* Penser que le LLM “voit” {context}

* Penser que le LLM récupère les documents lui-même

* Oublier que le prompt est entièrement reconstruit à chaque requête

**Règle d’or**

Le LLM ne voit jamais LangChain.
Il ne voit qu’un texte final, produit par PromptTemplate.format().